In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from datetime import date

In [2]:
def merge_sort_orders(df):
    if len(df) <= 1:
        return df
    mid = len(df) // 2
    left_half = df.iloc[:mid]
    right_half = df.iloc[mid:]
    left_half_sorted = merge_sort_orders(left_half)
    right_half_sorted = merge_sort_orders(right_half)
    return merge(left_half_sorted, right_half_sorted)

def merge(left, right):
    result = pd.DataFrame(columns=left.columns)
    left_index, right_index = 0, 0
    while left_index < len(left) and right_index < len(right):
        if left.iloc[left_index]['delivery_date'] < right.iloc[right_index]['delivery_date']:
            result = pd.concat([result, left.iloc[[left_index]]])
            left_index += 1
        elif left.iloc[left_index]['delivery_date'] == right.iloc[right_index]['delivery_date']:
            if left.iloc[left_index]['location'] < right.iloc[right_index]['location']:
                result = pd.concat([result, left.iloc[[left_index]]])
                left_index += 1
            else:
                result = pd.concat([result, right.iloc[[right_index]]])
                right_index += 1
        else:
            result = pd.concat([result, right.iloc[[right_index]]])
            right_index += 1
    while left_index < len(left):
        result = pd.concat([result, left.iloc[[left_index]]])
        left_index += 1
    while right_index < len(right):
        result = pd.concat([result, right.iloc[[right_index]]])
        right_index += 1
    return result

In [5]:
data = pd.read_excel('delivery_details.xlsx')
df = pd.DataFrame(data)
df['delivery_date'] = pd.to_datetime(df['delivery_date'])
sorted_df = merge_sort_orders(df)
print(sorted_df)

    order_id customer_name delivery_date item_weight location        street
136      172        Andrew    2024-05-11          20  Chennai      Santhome
135      171       William    2024-05-11          14  Chennai      Santhome
134      170          Alex    2024-05-11          20  Chennai      Santhome
133      167          Levi    2024-05-11          15  Chennai      Santhome
131      162         Lucas    2024-05-11          12  Chennai  Thoraipakkam
..       ...           ...           ...         ...      ...           ...
15        16        Andrew    2024-05-12          16  Madurai     SS colony
14        15       William    2024-05-12          13  Madurai     SS colony
12        13           Joe    2024-05-12          19  Madurai     SS colony
9         10        Andrew    2024-05-12           5  Madurai     SS colony
1          2          Alex    2024-05-12          10  Madurai     SS colony

[151 rows x 6 columns]


In [6]:
def knapsack(truck_capacity, items):
    n = len(items)
    K = [[0 for _ in range(truck_capacity + 1)] for _ in range(n + 1)]

    for i in range(n + 1):
        for w in range(truck_capacity + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif items[i - 1]['item_weight'] <= w:
                K[i][w] = max(items[i - 1]['item_weight'] + K[i - 1][w - items[i - 1]['item_weight']], K[i - 1][w])
            else:
                K[i][w] = K[i - 1][w]

    result = []
    i, w = n, truck_capacity
    while i > 0 and w > 0:
        if K[i][w] != K[i - 1][w]:
            result.append(items[i - 1])
            w -= items[i - 1]['item_weight']
        i -= 1


    return result[::-1]

In [7]:
truck_list=[]
today = date.today()
print(today)
temp_data = sorted_df[sorted_df['delivery_date']==str(today)]
locations = sorted_df['location'].unique()
for j in range(len(locations)):
    truck_details = pd.read_excel('truck_details.xlsx')
    temp_data2=temp_data[temp_data['location']==locations[j]]
    remaining_orders = temp_data2.to_dict('records')
    for truck, capacity in zip(truck_details['Truck'], truck_details['Truck_Capacity']):
        if truck in truck_list:
            continue
        truck_capacity = capacity
        items = [order for order in remaining_orders if order['item_weight'] <= truck_capacity]
        if not items:
            continue
        knapsack_items = knapsack(truck_capacity, items)
        total_weight = sum(item['item_weight'] for item in knapsack_items)
        print(f"Truck: {truck}, Total Weight: {total_weight}")
        print("Items to be loaded into the truck:")
        for item in knapsack_items:
            truck_list.append(truck)
            print(f"Order ID: {item['order_id']}, Weight: {item['item_weight']}, Location: {item['location']}, Delivery Date: {item['delivery_date']}")
            temp_data.loc[temp_data['order_id']==item['order_id'],'Truck'] = truck
        print()
        remaining_orders = [order for order in remaining_orders if order not in knapsack_items]
temp_data.to_excel('Truck Allocation.xlsx', index=False)
# myfile="Truck Allocation.xlsx"
# if os.path.isfile(myfile):
#     os.remove(myfile)

# for i in range(len(truck_list)):
#     truck_details["Allocation"].fillna("Not Allocated", inplace = True)

2024-05-11
Truck: Truck1, Total Weight: 77
Items to be loaded into the truck:
Order ID: 172, Weight: 20, Location: Chennai, Delivery Date: 2024-05-11 00:00:00
Order ID: 162, Weight: 12, Location: Chennai, Delivery Date: 2024-05-11 00:00:00
Order ID: 156, Weight: 19, Location: Chennai, Delivery Date: 2024-05-11 00:00:00
Order ID: 148, Weight: 7, Location: Chennai, Delivery Date: 2024-05-11 00:00:00
Order ID: 141, Weight: 19, Location: Chennai, Delivery Date: 2024-05-11 00:00:00

Truck: Truck2, Total Weight: 73
Items to be loaded into the truck:
Order ID: 171, Weight: 14, Location: Chennai, Delivery Date: 2024-05-11 00:00:00
Order ID: 170, Weight: 20, Location: Chennai, Delivery Date: 2024-05-11 00:00:00
Order ID: 167, Weight: 15, Location: Chennai, Delivery Date: 2024-05-11 00:00:00
Order ID: 130, Weight: 7, Location: Chennai, Delivery Date: 2024-05-11 00:00:00
Order ID: 129, Weight: 8, Location: Chennai, Delivery Date: 2024-05-11 00:00:00
Order ID: 124, Weight: 9, Location: Chennai, De

<ipython-input-7-275bbb96ce3c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data.loc[temp_data['order_id']==item['order_id'],'Truck'] = truck


Truck: Truck7, Total Weight: 80
Items to be loaded into the truck:
Order ID: 190, Weight: 15, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00
Order ID: 187, Weight: 11, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00
Order ID: 185, Weight: 12, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00
Order ID: 184, Weight: 8, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00
Order ID: 161, Weight: 17, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00
Order ID: 159, Weight: 17, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00

Truck: Truck8, Total Weight: 76
Items to be loaded into the truck:
Order ID: 182, Weight: 19, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00
Order ID: 181, Weight: 7, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00
Order ID: 118, Weight: 11, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00
Order ID: 108, Weight: 15, Location: Coimbatore, Delivery Date: 2024-05-11 00:00:00
Order ID: 103, Weight: 6, L

In [8]:
G = nx.DiGraph()

G.add_edge('Chennai', 'Villupuram', distance=160, time=3)
G.add_edge('Villupuram', 'Tiruchirappalli', distance=160, time=3.5)
G.add_edge('Tiruchirappalli', 'Dindigul', distance=100, time=2)
G.add_edge('Dindigul', 'Madurai', distance=35, time=1)
G.add_edge('Chennai', 'Chengalpattu', distance=55, time=1)
G.add_edge('Chengalpattu', 'Tindivanam', distance=80, time=1)
G.add_edge('Tindivanam','Villupuram', distance=35, time=1)
G.add_edge('Villupuram','Ulundurpet', distance=70, time=1)
G.add_edge('Ulundurpet','Tiruchirappalli', distance=110, time=2)
G.add_edge('Villupuram','Pondicherry', distance=40, time=1)
G.add_edge('Pondicherry','Cuddalore',distance=25,time=1)
G.add_edge('Cuddalore','Chidambaram',distance=65, time=1)
G.add_edge('Chidambaram','Nagapattinam', distance=70, time=1)
G.add_edge('Nagapattinam','Thanjavur', distance=90, time=1)
G.add_edge('Thanjavur','Tiruchirappalli', distance=60, time=1)

G.add_edge('Villupuram','Salem', distance=170, time=3)
G.add_edge('Salem','Coimbatore', distance=170, time=3)
G.add_edge('Ulundurpet','Salem', distance=140, time=3)
G.add_edge('Chennai','Tiruvannamalai', distance=190, time=3)
G.add_edge('Villupuram','Salem', distance=170, time=3)
G.add_edge('Madurai','Kanyakumari', distance=295, time=5)

G.add_edge('Chennai','Tambaram',distance=25,time=0.5)
G.add_edge('Tambaram','Chengalpattu',distance=30, time=40)
G.add_edge('Chengalpattu','Villupuram',distance=90, time=1.5)
G.add_edge('Madurai','Tirunelveli', distance=150, time=1.5)
G.add_edge('Tirunelveli','Nagercoil', distance=70, time=1.5)
G.add_edge('Nagercoil','Kanyakumari', distance=20, time=0.5)

G.add_edge('Chennai','Vellore',distance=140,time=2.5)
G.add_edge('Krishnagiri','Salem',distance=100,time=2)

G.add_edge('Vellore','Krishnagiri',distance=150,time=3)
G.add_edge('Salem','Namakkal',distance=55,time=1)
G.add_edge('Namakkal','Karur',distance=60,time=1.5)
G.add_edge('Karur','Dindigul',distance=85,time=2)
G.add_edge('Dindigul','Virudhunagar',distance=50,time=1)
G.add_edge('Virudhunagar','Kanyakumari',distance=320,time=6)
G.add_edge('Chennai','Pondicherry',distance=150,time=3)
G.add_edge('Pondicherry','Chidambaram',distance=110,time=2)
G.add_edge('Chidambaram','Kumbakonam',distance=65,time=1.5)
G.add_edge('Kumbakonam','Tirunelveli',distance=300,time=6)
G.add_edge('Chennai','Sriperumbudur',distance=45,time=1)
G.add_edge('Sriperumbudur','Vellore',distance=120,time=2.5)
G.add_edge('Krishnagiri','Salem',distance=100,time=2)

truck_allocate=pd.read_excel('Truck Allocation.xlsx')
locations = truck_allocate['location'].unique()
for j in range(len(locations)):
    distance_path = nx.shortest_path(G, source='Chennai', target=locations[j], weight='distance')
#     print("\nShortest path from chennai to ", locations[j],"\n\t", distance_path)
    print("\nShortest path from chennai to ", locations[j],"\n\t")
    print(end="\t")
    for i in range(len(distance_path)):
        if i==len(distance_path)-1:
            print(distance_path[i], end=' ')
        else:
            print(distance_path[i],' --> ', end=' ')
    total_distance = sum(G[u][v]['distance'] for u, v in zip(distance_path[:-1], distance_path[1:]))
    print("\n\tTotal distance:", total_distance, "km")

    time_path = nx.shortest_path(G, source='Chennai', target=locations[j], weight='time')
    # print("Shortest path by time:", time_path)
    total_time = sum(G[u][v]['time'] for u, v in zip(time_path[:-1], time_path[1:]))
    print("\tTotal time:", total_time, "hours")
#     truck_fil=truck_allocate[truck_allocate['location']==locations[j]]
#     truck_no = truck_fil['Truck'].unique()



Shortest path from chennai to  Chennai 
	
	Chennai 
	Total distance: 0 km
	Total time: 0 hours

Shortest path from chennai to  Coimbatore 
	
	Chennai  -->  Chengalpattu  -->  Villupuram  -->  Salem  -->  Coimbatore 
	Total distance: 485 km
	Total time: 8.5 hours

Shortest path from chennai to  Kanyakumari 
	
	Chennai  -->  Chengalpattu  -->  Villupuram  -->  Tiruchirappalli  -->  Dindigul  -->  Madurai  -->  Tirunelveli  -->  Nagercoil  -->  Kanyakumari 
	Total distance: 680 km
	Total time: 12.0 hours

Shortest path from chennai to  Madurai 
	
	Chennai  -->  Chengalpattu  -->  Villupuram  -->  Tiruchirappalli  -->  Dindigul  -->  Madurai 
	Total distance: 440 km
	Total time: 8.5 hours
